In [10]:
import os
import re
import pandas as pd

# Analiza resultados generales

In [39]:
def select_files(files, name="version 3 drought 2_5"):
    files = [f for f in files if name in f]
    return files


def count_significative_vars(s):
    is_p01  = s.str.contains("*", regex=False)
    is_p005 = s.str.contains("**", regex=False)
    is_p001 = s.str.contains("***", regex=False)
    
    is_negative  = s.str.contains("-.*\.", regex=True)

    count_positive_p01  = (is_p01 & ~is_negative).sum()
    count_positive_p005 = (is_p005 & ~is_negative).sum()
    count_positive_p001 = (is_p001 & ~is_negative).sum()
    
    return count_positive_p01, count_positive_p005, count_positive_p001 

def dict_to_df(dict):
    results_df = pd.DataFrame().from_dict(dict, orient="index", columns=["p=0.1", "p=0.05", "p=0.01"]).reset_index(names="Specification")
    return results_df

def process_significance(s):
    # Replace all values that does not contain an asterisk with NaN and turn them into floats
    s = s.where(s.str.contains("*", regex=False))
    return s

def make_float(s):
    # Remove asterisks and turn into floats
    s = s.replace(r'[^0-9\-.]', "", regex=True)
    return s

def parse_name(name):
    words = name.split(" ")
    
    version_nam, version_num = words[:2]
    months_num, months_nam = words[-2], words[-1]
    
    if version_num in ["3", "4"]:
        drought_nam, drought_num = words[2], words[3]

    else:
        drought_nam, drought_num = "", ""

    return version_nam, version_num, drought_nam, drought_num, months_num, months_nam

In [ ]:
import re

folder = r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Outputs"
files = os.listdir(folder)
files = [f for f in files if ".tex" in f]
assert len(files)!=0


for i, file in enumerate(files):

    filename = rf"{folder}\{file}"
    df = pd.read_csv(filename,
                    sep='&',
                    header=0,
                    skiprows=4,
                    skipfooter=3,
                    engine='python')

    keywords = "drought|excessiverain|temp_|spi" # Regex allowed
    
    df = df.set_index(df.columns[0]) # Set variable names in index
    df = df[df.index != ' '] # Remove standard errors
    df = df[df.index.str.contains(f'{keywords}|VARIABLES')]

    results = {}
    for specification in df.columns:
        counts = count_significative_vars(df[specification])
        specification = re.sub(r'[^0-9a-zA-Z]+', '', specification)
        results[specification] = counts

    results_df = dict_to_df(results)    
    results_df["path"] = filename
    results_df["file"] = file
    results_df = results_df[['file', 'path', 'Specification', 'p=0.1', 'p=0.05', 'p=0.01']]

    if i==0:
        results_df.to_csv("regs_significance_all_models.csv", index=False)    
    else:
        results_df.to_csv("regs_significance_all_models.csv", mode="a", index=False, header=False)

# Resultados por país

In [78]:
import plotly.graph_objects as go

def dot_plot(all_specs, varname, quarter):
    
    fig = go.Figure()

    colors = [
        'rgba(156, 165, 196, 0.95)','rgba(156, 165, 196, 0.95)','rgba(156, 165, 196, 0.95)','rgba(156, 165, 196, 0.95)',
        'rgba(204, 204, 204, 0.95)','rgba(204, 204, 204, 0.95)','rgba(204, 204, 204, 0.95)','rgba(204, 204, 204, 0.95)',
        'rgba(255, 166, 86, 0.95)','rgba(255, 166, 86, 0.95)','rgba(255, 166, 86, 0.95)','rgba(255, 166, 86, 0.95)',
        'rgba(240, 230, 140, 0.95)','rgba(240, 230, 140, 0.95)','rgba(240, 230, 140, 0.95)','rgba(240, 230, 140, 0.95)',
    ]
    data = all_specs[
        all_specs.variable.str.contains("drought") &
        all_specs.variable.str.contains(quarter)
    ]
    for i, spec in enumerate(data.specification.unique()):
        data_plot = data[data.specification == spec]
        fig.add_trace(go.Scatter(
            x=data_plot["value"].astype("float"),
            y=data_plot["Country"],
            marker=dict(
                color=colors[i],
                size=16
            ),
            mode='markers',
            name=spec,
        ))

    fig.update_traces(mode='markers', marker=dict(line_width=1, symbol='circle', size=16))

    fig.update_layout(
        title=varname,
        xaxis=dict(
            showgrid=False,
            showline=True,
            linecolor='rgb(102, 102, 102)',
            tickfont_color='rgb(102, 102, 102)',
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
        ),
        margin=dict(l=140, r=40, b=50, t=80),
        legend=dict(
            font_size=10,
            yanchor='top',
            xanchor='center',
        ),
        width=1600,
        height=600,
        paper_bgcolor='white',
        plot_bgcolor='white',
        hovermode='closest',
    )
    
    # Add a vertical line at zero
    fig.update_layout(shapes=[
        dict(
            type="line",
            x0=0,
            y0=0,
            x1=0,
            y1=all_specs.Country.drop_duplicates().shape[0],
            line=dict(
                color="red",
                width=2,
                dash="dashdot",
            )
        )
    ])

    fig.show()

In [81]:
import re
from tqdm import tqdm

folder = r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Outputs\countries"
name = "version 3 drought 2_5 12 months"
version_nam, version_num, drought_nam, drought_num, months_num, months_nam = parse_name(name)

files = os.listdir(folder)
files = [f for f in files if ".tex" in f]
files = select_files(files, name=name)

all_specs = []
for file in tqdm(files):

    filename = rf"{folder}\{file}"
    country = re.search(r"([A-Z]{3})", file).group(0)
    
    df = pd.read_csv(filename,
                    sep='&',
                    header=0,
                    skiprows=4,
                    skipfooter=3,
                    engine='python')

    keywords = "temp|spi|drought|excessiverain|prec" # Regex allowed
    
    # Process data
    df = df.set_index(df.columns[0]) # Set variable names in index
    df = df[df.index != ' '] # Remove standard errors
    df = df[df.index.str.contains(keywords)]
    df.index = df.index.str.strip()
    df.index.name = "variable"
    df = df.reset_index()
    results = {}
    df.columns = [re.sub(r'[^0-9a-zA-Z]+', '', col) for col in df.columns]

    # Format dataframe
    drought_num_str = drought_num.replace("_", r"\_")
    months_num_str = fr"\_{months_num}\_"
    country_specs = df.melt(id_vars=["variable"], var_name="specification", value_name="value")
    country_specs["value"] = process_significance(country_specs.value)    
    country_specs["value"] = make_float(country_specs.value)    
    country_specs["Country"] = country
    country_specs["variable"] = country_specs["variable"].str.replace(months_num_str, " ").str.replace(drought_num_str, drought_num)
    all_specs += [country_specs]
    
all_specs = pd.concat(all_specs)

dot_plot(all_specs, f"drought", "q1")
dot_plot(all_specs, f"drought", "q2")
dot_plot(all_specs, f"drought", "q3")
dot_plot(all_specs, f"drought", "30d")
dot_plot(all_specs, f"drought", "30d3m")
dot_plot(all_specs, f"drought", "3m6m")
dot_plot(all_specs, f"drought", "6m12m")

 17%|█▋        | 9/54 [00:00<00:00, 83.24it/s]C:\Users\ofici\AppData\Local\Temp\ipykernel_24444\3964789092.py:30: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\ofici\AppData\Local\Temp\ipykernel_24444\3964789092.py:30: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

 54%|█████▎    | 29/54 [00:00<00:00, 85.37it/s]C:\Users\ofici\AppData\Local\Temp\ipykernel_24444\3964789092.py:30: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly cal